In [ ]:
import regex as re

# Implement opcodes

In [ ]:
# Addition
def addr(reg, A, B, C):
    reg[C] = reg[A] + reg[B]
    
def addi(reg, A, B, C):
    reg[C] = reg[A] + B

# Multiplication
def mulr(reg, A, B, C):
    reg[C] = reg[A] * reg[B]
    
def muli(reg, A, B, C):
    reg[C] = reg[A] * B
    
# Bitwise-AND
def banr(reg, A, B, C):
    reg[C] = reg[A] & reg[B]
    
def bani(reg, A, B, C):
    reg[C] = reg[A] & B
    
# Bitwise-OR
def borr(reg, A, B, C):
    reg[C] = reg[A] | reg[B]
    
def bori(reg, A, B, C):
    reg[C] = reg[A] | B
    
# Assignment
def setr(reg, A, B, C):
    reg[C] = reg[A]
    
def seti(reg, A, B, C):
    reg[C] = A
    
# Greater-than testing
def gtir(reg, A, B, C):
    if A > reg[B]: reg[C] = 1
    else: reg[C] = 0

def gtri(reg, A, B, C):
    if reg[A] > B: reg[C] = 1
    else: reg[C] = 0

def gtrr(reg, A, B, C):
    if reg[A] > reg[B]: reg[C] = 1
    else: reg[C] = 0
        
# Equality testing
def eqir(reg, A, B, C):
    if A == reg[B]: reg[C] = 1
    else: reg[C] = 0

def eqri(reg, A, B, C):
    if reg[A] == B: reg[C] = 1
    else: reg[C] = 0

def eqrr(reg, A, B, C):
    if reg[A] == reg[B]: reg[C] = 1
    else: reg[C] = 0
        
# Store a list of all the opcodes
opcodes = [
    addr, addi, mulr, muli,
    banr, bani, borr, bori,
    setr, seti,
    gtir, gtri, gtrr,
    eqir, eqri, eqrr
]

# Also create a lookup by name since we're going to need this
opcodes_by_name = { f.__name__: f for f in opcodes }


In [ ]:

def test(before, sample, after):
    """ 
        Tests a program sample to see which opcodes produces the observed output. 
        Returns possible opcodes by name.
    """
    valid = []
    for op in opcodes:
        reg = before.copy()
        op(reg, *sample[1:])
        if (reg == after):
            valid.append(op.__name__)
    return valid


In [ ]:
test([3,2,1,1], [9,2,1,2], [3,2,2,1])

In [ ]:
with open("./16-input.txt", "r") as FILE:
    data = FILE.read()
    
# pattern = re.compile(r"BEFORE: [(.*)]\s+(\d+ \d+ \d+ \d+)\s+AFTER: [(.*)]", re.MULTILINE)
pattern = re.compile(r"Before:\s+\[(.*)\]\s+(\d+ \d+ \d+ \d+)\s+After:\s+\[(.*)\]", re.MULTILINE)

matches = pattern.findall(data)

count = 0
print(len(matches),"samples read")
for m in matches:
    before = [int(n.strip()) for n in m[0].split(',')]
    sample = [int(n.strip()) for n in m[1].split(' ')]
    after = [int(n.strip()) for n in m[2].split(',')]
    
    ops = test(before, sample, after)
    if len(ops) >= 3: 
        count+=1
    
print(count, "samples matched 3 or more operations")
    


# Part 2

Using the same approach, we now want to narrow down what the opcode is for each operation

In [ ]:
operations = dict()

for m in matches:
    before = [int(n.strip()) for n in m[0].split(',')]
    sample = [int(n.strip()) for n in m[1].split(' ')]
    after = [int(n.strip()) for n in m[2].split(',')]
    
    ops = test(before, sample, after)
    
    opcode = sample[0]
    possible_operations = operations.get(opcode, None)
    if possible_operations is None:
        possible_operations = set(ops)
    else:
        possible_operations = possible_operations & set(ops)
    
    operations[opcode] = possible_operations
    
# display(operations)

display("We will now reduce this to a simple set through elimination")

while max([len(x) for x in operations.values()]) > 1:
    for k, v in operations.items():
        if len(v) == 1:
            for k2, v2 in operations.items():
                if k != k2:
                    operations[k2] = v2 - v

operations = {k: v.pop() for k,v in operations.items()}
operations = {k: opcodes_by_name[v] for k,v in operations.items()}
display(operations)

In [ ]:
def execute_line(reg, opcode, A, B, C):
    reg = reg.copy()
    op = operations[opcode]
    op(reg, A, B, C)
    return reg


In [ ]:
# Let's do a simple test - let's run a simple program. Expect [1,2,3,4] at the end
# You may need to adjust the opcodes to match your input
reg = [0,0,0,0]
reg = execute_line(reg, 5, 1, 0, 0)  # setr(1,0,0) - Value 1 => Register 0
reg = execute_line(reg, 1, 0, 1, 1)  # addi(0,1,1) - Register 0 + Value 1 => Register 1
reg = execute_line(reg, 12, 0, 1, 2) # addr(0,1,2) - Register 0 + Register 1 => Register 2
reg = execute_line(reg, 4, 1, 1, 3)  # mulr(1,1,3) - Register 1 * Register 1 => Register 4

reg

We can now read the program line by line and execute. We find the start of the program by looking for three blank lines.

In [ ]:
program = data[data.index("\n\n\n"):].strip().splitlines()
reg = [0,0,0,0]
for line in program:
    line = [int(x.strip()) for x in line.split(' ')]
    reg = execute_line(reg, *line)

reg
